Use kernal conda_tensorflow_p36

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

plt.style.use('ggplot')

Using TensorFlow backend.


## Tom Brady

In [2]:
# Read in data
brady = pd.read_csv('../data/data_final/final/features_raw/brady.csv')

# Select features for RNN, along with target. Specifically, remove averages.
features = [
    'target',
    'date',
    'player_home',
    'betting_over_under_line',
    'betting_spread',
    'weather_temperature',
    'weather_wind_mph',
    'weather_humidity',
    'weather_inclement',
    'defense_fumbles_forced_moving_avg_1',
    'defense_fumbles_forced_moving_avg_3',
    'defense_fumbles_forced_moving_avg_5',
    'defense_interceptions_moving_avg_1',
    'defense_interceptions_moving_avg_3',
    'defense_interceptions_moving_avg_5',
    'defense_pass_yards_allowed_moving_avg_1',
    'defense_pass_yards_allowed_moving_avg_3',
    'defense_pass_yards_allowed_moving_avg_5',
    'defense_rush_yards_allowed_moving_avg_1',
    'defense_rush_yards_allowed_moving_avg_3',
    'defense_rush_yards_allowed_moving_avg_5',
    'defense_sacks_moving_avg_1',
    'defense_sacks_moving_avg_3',
    'defense_sacks_moving_avg_5',
    'defense_points_allowed_moving_avg_1',
    'defense_points_allowed_moving_avg_3',
    'defense_points_allowed_moving_avg_5',
    'twitter_pct_player_tweets',
    'twitter_pct_opponent_tweets',
    'twitter_count_player_swing_1_3',
    'twitter_count_opponent_swing_1_3',
    'twitter_player_net_sentiment',
    'twitter_opponent_net_sentiment',
    'twitter_player_pct_neutral',
    'twitter_opponent_pct_neutral',
    'twitter_net_sentiment_player_swing_1_3',
    'twitter_net_sentiment_opponent_swing_1_3',
    'twitter_pct_neutral_player_swing_1_3',
    'twitter_pct_neutral_opponent_swing_1_3'
]

brady = brady.loc[:, features]
brady['date'] = pd.to_datetime(brady['date'])

In [3]:
# For each column in X, replace NaNs with training set naive Bayes
def replace_nans_mean(df, split):
    df_cp = df.copy()
    for col in df_cp.columns:
        if col == 'date':
            continue
        naive_estimate = df_cp.loc[:split, col].mean()
        df_cp.loc[pd.isnull(df_cp[col]), col] = naive_estimate
        
    return df_cp
        
brady = replace_nans_mean(brady, 32)

In [4]:
# Train + test set indices
split_date = pd.Timestamp(2012,5,1)
train_idx = brady.loc[brady['date']<split_date].index.values
test_idx = brady.loc[brady['date']>split_date].index.values

# Scale Features
brady.drop('date', axis=1, inplace=True)
scaler = MinMaxScaler()
scaler.fit(brady)
brady = scaler.transform(brady)

# Train + test sets
train, test = brady[train_idx], brady[test_idx]

# Split to X and Y
# X_train, X_test, y_train, y_test = X, y = brady.iloc[:,1:], brady.iloc[:,0]

/home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
def build_lstm_data(df, window):
    X = []
    y = []
    for i in range(len(df)-window):
        temp_set = df[i:(i+window)].copy()
        X.append(temp_set[:,1:])
        y.append(temp_set[window-1,0])

    return np.array(X), np.array(y)

In [6]:
# Create train and test sets
window = 5
X, y = build_lstm_data(brady, window)
X_train, y_train = X[(train_idx-window)[window:]], y[(train_idx-window)[window:]]
X_test, y_test = X[(test_idx-window)], y[(test_idx-window)]

In [7]:
# Invert scaling of y for testing comparison
y_inv = np.concatenate((y_test.reshape(len(y_test),1), test[:,1:]), axis=1)
y_inv = scaler.inverse_transform(y_inv)
y_inv = y_inv[:,0]

In [ ]:
%%time
# Build networks. Loop n times to deal with ranodmness.
n_loops = 10
rmses, maes = [], []
best_rmse, best_mae = 100, 100
best_model = None
best_preds = []

for i in range(0, n_loops):
    model = Sequential()
    model.add(LSTM(30, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(.3))
    model.add(LSTM(30))
    model.add(Dropout(.3))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    
    # Fit network
    lstm_model = model.fit(X_train, y_train, epochs=1000, batch_size=5, validation_data=(X_test, y_test), verbose=2, shuffle=False)
    
    # Make predictions for test set
    preds = model.predict(X_test)
    
    # Invert scaling for preds
    preds_inv = np.concatenate((preds, test[:,1:]), axis=1)
    preds_inv = scaler.inverse_transform(preds_inv)
    preds_inv = preds_inv[:,0]
    
    # Calculate RMSE and MAE
    rmses.append(np.sqrt(mean_squared_error(y_inv, preds_inv)))
    maes.append(mean_absolute_error(y_inv, preds_inv))
    
    if maes[i] < best_mae:
        best_rmse = rmses[i]
        best_mae = maes[i]
        best_model = lstm_model
        best_preds = preds_inv
    

W1106 00:52:42.651103 140091925624640 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1106 00:52:44.377763 140091925624640 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1106 00:52:46.134674 140091925624640 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1106 00:52:48.412560 140091925624640 depr

Train on 26 samples, validate on 16 samples
Epoch 1/1000
 - 37s - loss: 0.3163 - val_loss: 0.2038
Epoch 2/1000
 - 1s - loss: 0.2582 - val_loss: 0.2317
Epoch 3/1000
 - 1s - loss: 0.2673 - val_loss: 0.2385
Epoch 4/1000
 - 1s - loss: 0.2442 - val_loss: 0.2142
Epoch 5/1000
 - 1s - loss: 0.2338 - val_loss: 0.2085
Epoch 6/1000
 - 1s - loss: 0.2092 - val_loss: 0.2093
Epoch 7/1000
 - 0s - loss: 0.2460 - val_loss: 0.2112
Epoch 8/1000
 - 1s - loss: 0.2323 - val_loss: 0.2129
Epoch 9/1000
 - 1s - loss: 0.2267 - val_loss: 0.2147
Epoch 10/1000
 - 1s - loss: 0.2646 - val_loss: 0.2193
Epoch 11/1000
 - 1s - loss: 0.2538 - val_loss: 0.2231
Epoch 12/1000
 - 1s - loss: 0.2466 - val_loss: 0.2164
Epoch 13/1000
 - 1s - loss: 0.2371 - val_loss: 0.2155
Epoch 14/1000
 - 1s - loss: 0.2142 - val_loss: 0.2103
Epoch 15/1000
 - 1s - loss: 0.2206 - val_loss: 0.2050
Epoch 16/1000
 - 1s - loss: 0.2221 - val_loss: 0.2037
Epoch 17/1000
 - 1s - loss: 0.2391 - val_loss: 0.2110
Epoch 18/1000
 - 1s - loss: 0.2353 - val_loss:

 - 0s - loss: 0.1295 - val_loss: 0.2104
Epoch 152/1000
 - 0s - loss: 0.1124 - val_loss: 0.2749
Epoch 153/1000
 - 0s - loss: 0.1091 - val_loss: 0.2171
Epoch 154/1000
 - 0s - loss: 0.0902 - val_loss: 0.2377
Epoch 155/1000
 - 0s - loss: 0.0983 - val_loss: 0.2445
Epoch 156/1000
 - 0s - loss: 0.0730 - val_loss: 0.2655
Epoch 157/1000
 - 0s - loss: 0.1078 - val_loss: 0.2128
Epoch 158/1000
 - 0s - loss: 0.1167 - val_loss: 0.2330
Epoch 159/1000
 - 0s - loss: 0.1303 - val_loss: 0.2738
Epoch 160/1000
 - 0s - loss: 0.1119 - val_loss: 0.2757
Epoch 161/1000
 - 0s - loss: 0.1180 - val_loss: 0.2142
Epoch 162/1000
 - 0s - loss: 0.1241 - val_loss: 0.2329
Epoch 163/1000
 - 0s - loss: 0.1084 - val_loss: 0.2329
Epoch 164/1000
 - 0s - loss: 0.1279 - val_loss: 0.2240
Epoch 165/1000
 - 0s - loss: 0.0941 - val_loss: 0.2348
Epoch 166/1000
 - 0s - loss: 0.1079 - val_loss: 0.2252
Epoch 167/1000
 - 0s - loss: 0.0988 - val_loss: 0.2365
Epoch 168/1000
 - 0s - loss: 0.1101 - val_loss: 0.2047
Epoch 169/1000
 - 0s - lo

 - 0s - loss: 0.0737 - val_loss: 0.2017
Epoch 301/1000
 - 0s - loss: 0.0586 - val_loss: 0.2108
Epoch 302/1000
 - 0s - loss: 0.0734 - val_loss: 0.2318
Epoch 303/1000
 - 0s - loss: 0.0685 - val_loss: 0.2031
Epoch 304/1000
 - 0s - loss: 0.0759 - val_loss: 0.2099
Epoch 305/1000
 - 0s - loss: 0.0799 - val_loss: 0.2460
Epoch 306/1000
 - 0s - loss: 0.0764 - val_loss: 0.2047
Epoch 307/1000
 - 0s - loss: 0.0629 - val_loss: 0.2058
Epoch 308/1000
 - 0s - loss: 0.0597 - val_loss: 0.2353
Epoch 309/1000
 - 0s - loss: 0.0786 - val_loss: 0.2298
Epoch 310/1000
 - 0s - loss: 0.0830 - val_loss: 0.2144
Epoch 311/1000
 - 0s - loss: 0.0687 - val_loss: 0.2438
Epoch 312/1000
 - 0s - loss: 0.0888 - val_loss: 0.2115
Epoch 313/1000
 - 0s - loss: 0.0725 - val_loss: 0.1958
Epoch 314/1000
 - 0s - loss: 0.0732 - val_loss: 0.1991
Epoch 315/1000
 - 0s - loss: 0.0721 - val_loss: 0.2199
Epoch 316/1000
 - 0s - loss: 0.0789 - val_loss: 0.2128
Epoch 317/1000
 - 0s - loss: 0.0656 - val_loss: 0.2022
Epoch 318/1000
 - 0s - lo

 - 1s - loss: 0.0642 - val_loss: 0.2328
Epoch 450/1000
 - 1s - loss: 0.0700 - val_loss: 0.2515
Epoch 451/1000
 - 1s - loss: 0.0635 - val_loss: 0.2244
Epoch 452/1000
 - 1s - loss: 0.0672 - val_loss: 0.2135
Epoch 453/1000
 - 1s - loss: 0.0618 - val_loss: 0.2387
Epoch 454/1000
 - 1s - loss: 0.0576 - val_loss: 0.2354
Epoch 455/1000
 - 1s - loss: 0.0551 - val_loss: 0.2184
Epoch 456/1000
 - 1s - loss: 0.0599 - val_loss: 0.2324
Epoch 457/1000
 - 1s - loss: 0.0567 - val_loss: 0.2379
Epoch 458/1000
 - 1s - loss: 0.0592 - val_loss: 0.2289
Epoch 459/1000
 - 1s - loss: 0.0438 - val_loss: 0.2339
Epoch 460/1000
 - 1s - loss: 0.0503 - val_loss: 0.2176
Epoch 461/1000
 - 1s - loss: 0.0589 - val_loss: 0.2198
Epoch 462/1000
 - 1s - loss: 0.0625 - val_loss: 0.2349
Epoch 463/1000
 - 1s - loss: 0.0641 - val_loss: 0.2247
Epoch 464/1000
 - 1s - loss: 0.0477 - val_loss: 0.2076
Epoch 465/1000
 - 1s - loss: 0.0952 - val_loss: 0.2118
Epoch 466/1000
 - 1s - loss: 0.0507 - val_loss: 0.2396
Epoch 467/1000
 - 1s - lo

In [ ]:
# Plot loss curve for best model
plt.rcParams['figure.figsize'] = (10, 5)

plt.plot(best_model.history['loss'], label='train')
plt.plot(best_model.history['val_loss'], label='test')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# Print RMSES and MAES
print('Best RMSE: {:.2f}'.format(best_rmse))
print('Best MAE: {:.2f}'.format(best_mae))
print('Average RMSE: {:.2f}'.format(np.mean(rmses)))
print('Average MAE: {:.2f}'.format(np.mean(maes)))

In [ ]:
plt.boxplot([maes, rmses])

In [ ]:
plt.rcParams['figure.figsize'] = (10, 5)
fig, ax = plt.subplots()
plot1 = ax.plot(y_inv, alpha = 0.8, label='actual')
plot1 = ax.plot(best_preds, alpha = 0.8, label='predicted')
ax.legend()
plt.xlabel('Week (2012)')
plt.ylabel('Fantasy Points')
plt.title('Tom Brady Fantasy Points 2012, Predicted vs Actual')

##### Iterative Training

In [42]:
# Rebuild network for iterative training - stateful model with one batch
model = Sequential()
model.add(LSTM(50, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

for i in range(50):
    model.fit(X_train, y_train, epochs=1, batch_size=1, verbose=0, shuffle=False)
    model.reset_states()

In [43]:
preds = []
for i in range(len(y_test)):
    X_test_formatted = X_test[i].reshape(X_test[i].shape[0], 1, X_test[i].shape[1])
    preds.append(model.predict(X_test_formatted)[0])
    X_train_upd = np.append(X_train, X_test_formatted, axis=0)
    y_train_upd = np.append(y_train, y_test[i])
    model.fit(X_train_upd, y_train_upd, epochs=10, batch_size=1, verbose=0, shuffle=False)
    model.reset_states()

In [44]:
preds_inv = np.concatenate((preds, X_test_orig), axis=1)
preds_inv = scaler.inverse_transform(preds_inv)
preds_inv = preds_inv[:,0]

rmse = np.sqrt(mean_squared_error(y_inv, preds_inv))
mae = mean_absolute_error(y_inv, preds_inv)
print('RMSE: {:.2f}'.format(rmse))
print('MAE: {:.2f}'.format(mae))

RMSE: 8.38
MAE: 6.80


In [ ]:
plt.rcParams['figure.figsize'] = (10, 5)
fig, ax = plt.subplots()
bar1 = ax.bar(x = np.arange(0, len(y_inv)) - .25, height=y_inv, width=.4, alpha = 0.8, label='actual')
bar2 = ax.bar(x = np.arange(0, len(preds_inv)) + .25, height=preds_inv, width=.4, alpha = 0.8, label='predicted')
ax.legend()
plt.xlabel('Week (2012)')
plt.ylabel('Fantasy Points')
plt.title('Tom Brady Fantasy Points 2012, Predicted vs Actual (Iterative Training)')